In [ ]:
!pip install PyMuPDF
!pip install ace_tools_open

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
import ace_tools_open as tools
import fitz  # PyMuPDF para leitura de PDFs

In [ ]:
# Função para extrair texto de um arquivo PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text()
    return text

# Função para processar o texto extraído e organizar as informações
def process_decision_text(text):
    # Inicializar dicionário para armazenar os dados
    data = {}

    # Extração baseada em palavras-chave
    data['Juiz'] = extract_value(text, 'Juiz:')
    data['Réu'] = extract_value(text, 'Réu:')
    data['Classe Processual'] = extract_value(text, 'Classe Processual:')
    data['Comarca'] = extract_value(text, 'Comarca:')
    data['Decisão Tomada'] = extract_value(text, 'Decisão:')
    data['Pena'] = extract_value(text, 'Pena:')

    return data

# Função auxiliar para localizar valores no texto com base em palavras-chave
def extract_value(text, keyword):
    try:
        start_idx = text.index(keyword) + len(keyword)
        end_idx = text.index('\n', start_idx)  # Fim da linha
        return text[start_idx:end_idx].strip()
    except ValueError:
        return None

# Lista de arquivos PDF
pdf_files = [
    'doc_55662302.pdf',
    'doc_61089526.pdf',
    'doc_61406539.pdf',
    'doc_95012143.pdf',
    'doc_108959255.pdf'
]

# Lista para armazenar os dados extraídos
all_data = []

# Processar cada PDF
for pdf_file in pdf_files:
    text = extract_text_from_pdf(pdf_file)
    decision_data = process_decision_text(text)
    all_data.append(decision_data)

# Criar um DataFrame do pandas
df = pd.DataFrame(all_data)

# Exibir o DataFrame
print(df)

# Salvar em um arquivo CSV
df.to_csv("decisoes_judiciais_finais.csv", index=False)

In [ ]:
# Carregar o dataset
df = pd.read_csv('decisoes_judiciais_finais.csv')

# Exibir as primeiras linhas do dataset para entender seu formato
print("Dados originais:")
print(df.head())

In [ ]:
# Carregar o dataset
df = pd.read_csv('decisoes_judiciais_finais.csv')

# Transformações no dataset
# 1. Pré Processamento - Deixar apenas as Decisões de culpados, remover as colunas Réu e Decisão e anonimizar o juiz
mask = (df['Decisão Tomada'] != 'Culpado')
df = df.loc[~mask]

#Remoção das colunas
df = df.drop(columns=['Réu', 'Decisão Tomada'])

#Anonimizar a coluna Juiz
df["Juiz"] = df["Juiz"].apply(lambda x: f"{hash(x) % 10000}")

# 2. Converter a coluna "Pena" para valores numéricos (em meses) e categorizando pelos quartis
def convert_penalty_to_months(pena):
    if pena == "Nenhuma":
        return 0
    elif "anos" in pena:
        return int(pena.split()[0]) * 12  # Converter anos para meses
    elif "meses" in pena:
        return int(pena.split()[0])  # Manter como meses
    return 0

df["Pena"] = df["Pena"].apply(convert_penalty_to_months)

quartis = df["Pena"].quantile([0.25, 0.5, 0.75]).values

# Criando uma nova coluna categórica com base nos quartis
df["Pena_Categorica"] = pd.cut(
    df["Pena"],
    bins=[-1, quartis[0], quartis[1], quartis[2], df["Pena"].max()],
    labels=["<= 63 Meses", "> 63 e <= 108", ">108 e <= 153", "> 153"]
)

# Exibindo o DataFrame resultante
df[["Pena","Pena_Categorica"]]

# 3. Codificar variáveis categóricas usando One Hot Encoder
#label_encoders = {}
#for column in ["Juiz", "Classe Processual", "Comarca"]:
#    le = LabelEncoder()
#    df[column] = le.fit_transform(df[column])
#    label_encoders[column] = le  # Guardar os encoders para decodificação futura, se necessário

encoder = OneHotEncoder(sparse_output=False)  # Evita multicolinearidade
encoded_vars = encoder.fit_transform(df[["Juiz", "Comarca", "Classe Processual"]])

# Criando um DataFrame com as variáveis codificadas
encoded_columns = encoder.get_feature_names_out(["Juiz", "Comarca", "Classe Processual"])
df_encoded = pd.DataFrame(encoded_vars, columns=encoded_columns)

#df_encoded

# Concatenando as novas variáveis ao DataFrame original e removendo as colunas antigas
df = pd.concat([df.drop(columns=["Juiz", "Comarca","Classe Processual"]).reset_index(drop=True), df_encoded.reset_index(drop=True)], axis=1)

# Exibindo as primeiras linhas do DataFrame transformado
tools.display_dataframe_to_user(name="Dataset com One-Hot Encoding", dataframe=df)

# Salvar o dataset transformado para uso posterior
df.to_csv('decisoes_judiciais_anonimizadas.csv', index=False)

In [ ]:
# Aplicando Random Forest e visualizando a importância das variáveis
columns = list(df.columns)
columns.remove("Pena")
columns.remove("Pena_Categorica")

# Definindo os dados de entrada (features) e saída (target)
x = df[columns]
y = df["Pena_Categorica"]

# Dividindo os dados em conjunto de treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Criando o modelo Random Forest
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42, bootstrap=True, max_depth=None)
random_forest_model.fit(x_train, y_train)

# Fazendo previsões
y_pred = random_forest_model.predict(x_test)
y_test

# Avaliando a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy * 100:.2f}%")

# Extraindo importância das variáveis
feature_importances = random_forest_model.feature_importances_
features = x.columns

importances_df = pd.DataFrame({"Variavel": features, "Importancia": feature_importances*100}).sort_values(by="Importancia", ascending=False)
importances_df

In [ ]:
#Importancia das variáveis agregadas
print("Importancia da Comarca",importances_df.query('Variavel.str.contains("Comarca")', engine='python').sum())
print("Importancia do Juiz",importances_df.query('Variavel.str.contains("Juiz")', engine='python').sum())
print("Importancia da Classe Processual",importances_df.query('Variavel.str.contains("Classe Processual")', engine='python').sum())

In [ ]:
# Visualizando a importância das variáveis
plt.figure(figsize=(10, 6))
plt.barh(importances_df['Variavel'], importances_df['Importancia'], color='blue')
plt.xlabel('Importância')
plt.title('Importância da Variável')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Plotar as árvores de decisão (apenas as primeiras 3 para demonstração)
plt.figure(figsize=(100,100))
for i in range(min(3, len(random_forest_model.estimators_))):  # Limitar a 3 árvores ou o número de árvores no modelo
    plt.subplot(1, min(3, len(random_forest_model.estimators_)), i + 1)
    plot_tree(random_forest_model.estimators_[i], feature_names=features, filled=True, rounded=True, class_names=np.unique(y).astype(str))
    plt.title(f"Árvore {i+1}")

plt.tight_layout()
plt.show()